In [ ]:
using CairoMakie;
using CSV, DataFrames

import LilGuys as lguys

using Arya

In [ ]:
model_dir = "/astro/dboyea/sculptor/orbits/orbit1/1e6/V32_r5"

In [ ]:
starsfile = "/astro/dboyea/sculptor/isolation/1e6/stars/exp2d_rs0.13_stars.hdf5"

In [ ]:
prob_df = lguys.load_hdf5_table(starsfile)

In [ ]:
out = lguys.Output(model_dir * "/out", weights=prob_df.probability)

In [ ]:
out[1].x_cen

In [ ]:
function get_xy(snap, time)
    x = snap.positions[2, :] #.- snap.x_cen[2]
	y = snap.positions[3, :] #.- snap.x_cen[3]
    return x, y
end

In [ ]:
profiles = lguys.Profiles3D("$model_dir/out/profiles.hdf5");

In [ ]:
function calc_stars_hist(snap, bins)
    x, y = get_xy(snap, 0)
	ms = probabilities[snap.index]
	h = Arya.histogram2d(x, y, bins, weights=ms)
    return h.values
end

In [ ]:
r_max = 200

In [ ]:
bins = LinRange(-r_max, r_max, 100)

In [ ]:
function xy_axis()
    fig = Figure()
    ax = Axis(fig[1, 1],
        xlabel="y / kpc",
        ylabel="z / kpc",
        aspect=DataAspect())

    return fig, ax
end

## Animate dark matter

In [ ]:
using KernelDensity

In [ ]:
function calc_dm_hist(snap, bins)
    x, y = get_xy(snap, 0)
	h = Arya.histogram2d(x, y, bins)
    return h.values
end

In [ ]:
h = calc_dm_hist(out[1], bins)

In [ ]:
color_kwargs = (;
    colorrange = (1e-5 * maximum(h), maximum(h)),
    colorscale = log10,
    colormap = :magma
    )

In [ ]:
fig, ax = xy_axis()

h = calc_dm_hist(out[1], bins)
image!((-r_max, r_max), (-r_max, r_max), h; color_kwargs...)

fig

In [ ]:
fig, ax = xy_axis()

h = calc_dm_hist(out[end], bins)
image!((-r_max, r_max), (-r_max, r_max), h; color_kwargs...)

fig

In [ ]:
framerate = 10

In [ ]:
idxs = 1:10:length(out)

In [ ]:
fig, ax = xy_axis()

h = calc_dm_hist(out[1], bins)

p = image!((-r_max, r_max), (-r_max, r_max), h; color_kwargs...)


record(fig, "sculptor_dm.mp4", idxs, framerate = framerate) do i
    snap = out[i]
    h = calc_dm_hist(snap, bins)
    p[3] = h
    println(i)
end

fig

In [ ]:
pwd()

## Animate stars

In [ ]:
function calc_stars_hist(snap, bins)
    x, y = get_xy(snap, 0)
	h = Arya.histogram2d(x, y, bins, weights=snap.weights)
    return h.values
end

In [ ]:
h = calc_stars_hist(out[1], bins)

In [ ]:
color_kwargs = (;
    colorrange = (1e-10 * maximum(h), maximum(h)),
    colorscale = log10,
    colormap = :magma
    )

In [ ]:
fig, ax = xy_axis()

h = calc_stars_hist(out[1], bins)
image!((-r_max, r_max), (-r_max, r_max), h; color_kwargs...)

fig

In [ ]:
fig, ax = xy_axis()

h = calc_stars_hist(out[end], bins)
image!((-r_max, r_max), (-r_max, r_max), h; color_kwargs...)

fig

In [ ]:
framerate = 10

In [ ]:
idxs = 1:10:length(out)

In [ ]:
fig, ax = xy_axis()

h = calc_stars_hist(out[1], bins)

p = image!((-r_max, r_max), (-r_max, r_max), h; color_kwargs...)


record(fig, "sculptor_stars.mp4", idxs, framerate = framerate) do i
    snap = out[i]
    h = calc_stars_hist(snap, bins)
    p[3] = h
    println(i)
end

fig

## Animate density profile

In [ ]:
function density_axis(gs; kwargs...)
    ax = Axis(gs;
        xlabel="log r / kpc",
        ylabel=L"\log\rho",
        limits=(-2, 2, -10, 1),
        kwargs...
    )

    return ax
end

In [ ]:
function density_axis()
    fig = Figure()
    ax = density_axis(fig[1, 1])

    return fig, ax
end

In [ ]:
times = out.times[profiles.snapshot_index] * lguys.T2GYR

In [ ]:
fig, ax = density_axis()
i = 1
lines!(profiles[i].log_r, log10.(profiles[i].rho))
fig

In [ ]:
fig, ax = density_axis()
i = length(times)
p = lines!(profiles[i].log_r, log10.(profiles[i].rho))
fig

In [ ]:


fig, ax = density_axis()
framerate=30
i = 1

p = lines!(profiles[i].log_r, log10.(profiles[i].rho))

record(fig, "sculptor_dm_density.mp4", 2:length(times), framerate = framerate) do i
    ax.title = string(times[i])
    
    prof = profiles[i]
    p[1] = [Makie.Point2f(x, y) for (x,y) in zip(prof.log_r, log10.(prof.rho))]
    println(i)
end

fig

## Animate orbit

In [ ]:
function r_t_axis()
    fig = Figure()
    ax = Axis(fig[1,1],
        xlabel="time / Gyr",
        ylabel=L"$r_\textrm{mw}$ / kpc",
    )

    return fig, ax
end

In [ ]:
function r_t_axis(gs)
    ax = Axis(gs,
        xlabel="time / Gyr",
        ylabel=L"$r_\textrm{mw}$ / kpc",
    )

    return ax
end

In [ ]:
r_mean = lguys.calc_r(out.x_cen)

In [ ]:
fig, ax = r_t_axis()
lines!(out.times * lguys.T2GYR, r_mean)
vlines!([NaN])
fig

In [ ]:
function rho_orbit_axis()
    fig = Figure(size=(700, 300))
    ax_rho = density_axis(fig[1, 1])
    ax_r = r_t_axis(fig[1, 2])

    ax_r.limits=(0, 10, 0.9minimum(r_mean), 1.1maximum(r_mean))

    colsize!(fig.layout, 1, Auto(1))
    colsize!(fig.layout, 2, Auto(0.5))

    return fig, ax_rho, ax_r
end

In [ ]:
rho_orbit_axis()[1]

In [ ]:
fig, ax, ax_r = rho_orbit_axis()
framerate=30

i = 1

p_rho = lines!(ax, profiles[i].log_r, log10.(profiles[i].rho))
orbit_points = Observable(Point2f[(out.times[1] * lguys.T2GYR, r_mean[1])])


p_r_break = vlines!(ax, [NaN], color=:black, alpha=0.3, linestyle=:dash)
p_r = lines!(ax_r, orbit_points)

r_last = Observable(r_mean[1])
r_last_last = Observable(r_last[])
t_last_peri = Observable(NaN)

record(fig, "sculptor_dm_density.mp4", 2:length(times), framerate = framerate) do i    
    prof = profiles[i]
    
    p_rho[1] = [Makie.Point2f(x, y) for (x,y) in zip(prof.log_r, log10.(prof.rho))]

    idx = profiles.snapshot_index[i]
    r = r_mean[idx]
    new_point = Point2f(times[i], r)
    orbit_points[] = push!(orbit_points[], new_point)

    if r_last_last[] > r_last[] && r_last[] < r_mean[idx]
        t_last_peri[] = times[i-1]
    end

    #todo oom but could be better
    σv = prof.v_circ_max * lguys.V2KMS / 3
    
    if !isnan(t_last_peri[])
        dt = times[i] - t_last_peri[]
        p_r_break[1] = [log10(σv * dt)] # approx
    end

    r_last_last[] = r_last[]
    r_last[] = r_mean[idx]
    println(i)

end

fig

## Animate phase space